In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
years = list(range(2023, 2017, -1))

In [ ]:
session = requests.Session()

# Replace with your login credentials and the login URL
payload = {
    'username': '<uname>',
    'password': '<pwd>'
}
login_page_url = 'https://stathead.com/users/login.cgi?token=1&__hstc=218152582.ffc1803babda4560a6e1c515a5470632.1706293742957.1708048543953.1708712387309.10&__hssc=218152582.6.1708712387309&__hsfp=3370414631&_gl=1*1s7yczf*_ga*MTYyMTU4MDIxMS4xNzA2MjkzNzM4*_ga_80FRT7VJ60*MTcwODcxMjM4Mi4xNS4xLjE3MDg3MTM5NTMuMC4wLjA.&redirect_uri=https%3A//fbref.com/en/comps/9/Premier-League-Stats'
session.post(login_page_url, data = payload)
all_matches = []
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
for year in years:
    data = session.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    # print(links)
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    print(year)
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = session.get(team_url)
        # # print(0.)
        # print(data)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links_1 = soup.find_all('a')
        # print(1.)
        # print(links_1)
        links_1 = [l_1.get("href") for l_1 in links_1]
        # print(2.)
        # print(links_1)
        links_1 = [l_1 for l_1 in links_1 if l_1 and 'all_comps/shooting/' in l_1]
        # print(3.)
        # print(links_1)
        data = session.get(f"https://fbref.com{links_1[0]}")
        # print(4.)
        # print(data)
        shooting = pd.read_html(data.text, match="Shooting")[0]
        
        shooting.columns = shooting.columns.droplevel()
        # print(5.)
        # print(shooting)
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"] 
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(3)

In [ ]:
match_df = pd.concat(all_matches)
match_df.to_csv("matches.csv")